### Necessary Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import komm

plt.style.use("default")
plt.rcParams["figure.figsize"] = (6, 4)

### Defining System

In [ ]:
# System parameters
area = [-1500, 1500]  # area of the simulation m^2
M = 4  # number of users
N = 1  # number of base stations

eta = 4  # path loss exponent
Pt = np.linspace(0, 40, 20)  # transmit power in dBm
Pt_lin = 10 ** (Pt / 10) / 1000  # transmit power in linear scale

bandwidth = 1e6  # bandwidth in Hz
noise = -174 + 10 * np.log10(bandwidth)  # noise power in dBm
noise_lin = 10 ** (noise / 10) / 1000  # noise power in linear scale
K = 5 * 10**4  # number of bits to be transmitted

In [ ]:
from matplotlib.markers import MarkerStyle

# Create a network with base station at the origin
bs = np.array([[0, 0]])
d = np.array([1000, 500, 200, 50])  # distance from base station

# or let distance be random
# d = np.random.uniform(0, RADIUS, size=M)

theta = np.random.uniform(0, 2 * np.pi, size=M)
users = np.zeros((M, 2))

for i in range(M):
    x = d[i] * np.cos(theta[i])
    y = d[i] * np.sin(theta[i])
    users[i] = np.array([x, y])

# Plot the network
plt.scatter(bs[:, 0], bs[:, 1], label="Base station")
plt.scatter(
    users[:, 0],
    users[:, 1],
    label="Users",
    marker=MarkerStyle("x"),
    color="r",
)
plt.legend()
plt.grid()
plt.title("System Model")
plt.xlim([-1500, 1500])
plt.ylim([-1500, 1500])
plt.show()

In [ ]:
def euclidean_distance(x1, y1, x2, y2):
    distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance

distance_mat = np.zeros((M,), dtype=float)
for i in range(M):
    distance_mat[i] = euclidean_distance(users[i, 0], users[i, 1], bs[0, 0], bs[0, 1])

print("Distance Matrix:")
print(distance_mat)

In [ ]:
h_mat = np.zeros((M, K // 2), dtype=complex)

In [ ]:
# Generate Rayleigh fading coefficients for each user
h_mat = np.zeros((M, K // 2), dtype=complex)

for i in range(M):
    h_mat[i] = (
        np.sqrt(distance_mat[i] ** (-eta))
        * (np.random.randn(K // 2) + 1j * np.random.randn(K // 2))
        / np.sqrt(2)
    )

print("Channel Matrix:")
print(h_mat.mean(axis=1))

In [ ]:
noise_mat = (
    np.sqrt(noise_lin)
    * (np.random.randn(M, K // 2) + 1j * np.random.randn(M, K // 2))
    / np.sqrt(2)
)

print("Noise Matrix:")
print(noise_mat.mean(axis=1))

In [ ]:
# Generate random message bits for each user
x = np.random.randint(0, 2, size=(M, K))

print("Bits:")
print(x)

In [ ]:
# Define the QPSK constellation
QPSK = komm.PSKModulation(4, phase_offset=np.pi / 4)

# Convert the input bits to QPSK symbols
x_qpsk = np.zeros((M, K // 2), dtype=complex)
for i in range(M):
    x_qpsk[i] = QPSK.modulate(x[i])

print("QPSK Symbols:")
print(x_qpsk)

In [ ]:
# Define power allocation coefficients
alpha = np.array([0.7, 0.15, 0.1, 0.05])

In [ ]:
# Superposition coding
x_sc = (
    np.sqrt(alpha[0]) * x_qpsk[0]
    + np.sqrt(alpha[1]) * x_qpsk[1]
    + np.sqrt(alpha[2]) * x_qpsk[2]
    + np.sqrt(alpha[3]) * x_qpsk[3]
)

print("Superposition Coded Symbols:")
print(x_sc)

In [ ]:
# Compute the received signal
y = np.zeros((len(Pt), M, K // 2), dtype=complex)
for i in range(len(Pt)):
    y[i] = np.sqrt(Pt_lin[i]) * x_sc * h_mat + noise_mat

# Perform equalization
y_eq = np.zeros((len(Pt), M, K // 2), dtype=complex)
for i in range(len(Pt)):
    for j in range(M):
        y_eq[i, j] = y[i, j] / h_mat[j]

In [ ]:
x_hat = np.zeros((len(Pt), M, K), dtype=int)

for i in range(len(Pt)):
    # Perform decoding of signal at user 1
    x_hat[i, 0] = QPSK.demodulate(y_eq[i, 0])  # Direct decoding

    # Perform decoding of signal at user 2
    u1_hat = QPSK.demodulate(y_eq[i, 1])  # Decode user 1 signal
    u1_remod = QPSK.modulate(u1_hat)  # Remodulate user 1 signal
    rem_u1 = y_eq[i, 1] - (np.sqrt(alpha[0] * Pt_lin[i]) * u1_remod)
    x_hat[i, 1] = QPSK.demodulate(rem_u1)  # Decode user 2 signal

    # Perform decoding of signal at user 3
    u1_hat = QPSK.demodulate(y_eq[i, 2])  # Decode user 1 signal
    u1_remod = QPSK.modulate(u1_hat)  # Remodulate user 1 signal
    u2_hat = QPSK.demodulate(
        y_eq[i, 2] - (np.sqrt(alpha[0] * Pt_lin[i]) * u1_remod)
    )  # Decode user 2 signal
    u2_remod = QPSK.modulate(u2_hat)  # Remodulate user 2 signal
    x_hat[i, 2] = QPSK.demodulate(
        y_eq[i, 2]
        - (np.sqrt(alpha[0] * Pt_lin[i]) * u1_remod)
        - (np.sqrt(alpha[1] * Pt_lin[i]) * u2_remod)
    )  # Decode user 3 signal

    # Perform decoding of signal at user 4
    u1_hat = QPSK.demodulate(y_eq[i, 3])  # Decode user 1 signal
    u1_remod = QPSK.modulate(u1_hat)  # Remodulate user 1 signal
    u2_hat = QPSK.demodulate(
        y_eq[i, 3] - (np.sqrt(alpha[0] * Pt_lin[i]) * u1_remod)
    )  # Decode user 2 signal
    u2_remod = QPSK.modulate(u2_hat)  # Remodulate user 2 signal
    u3_hat = QPSK.demodulate(
        y_eq[i, 3]
        - (np.sqrt(alpha[0] * Pt_lin[i]) * u1_remod)
        - (np.sqrt(alpha[1] * Pt_lin[i]) * u2_remod)
    )  # Decode user 3 signal
    u3_remod = QPSK.modulate(u3_hat)  # Remodulate user 3 signal
    x_hat[i, 3] = QPSK.demodulate(
        y_eq[i, 3]
        - (np.sqrt(alpha[0] * Pt_lin[i]) * u1_remod)
        - (np.sqrt(alpha[1] * Pt_lin[i]) * u2_remod)
        - (np.sqrt(alpha[2] * Pt_lin[i]) * u3_remod)
    )  # Decode user 4 signal

print("Decoded Bits:")
print(x_hat)